In [6]:
folder_path = 'streamingatscale/capture/ktsas18eventhubs/streamingatscale-2'

StatementMeta(sasesssparkpool, 71, 7, Finished, Available)

In [7]:
# Removing the container name from the folder path

first_slash_index = folder_path.index('/')
folder_path = folder_path[first_slash_index:]

StatementMeta(sasesssparkpool, 71, 8, Finished, Available)

/capture/ktsas18eventhubs/streamingatscale-2

In [8]:
from delta.tables import DeltaTable
import os
from pyspark.sql.functions import col
import pyspark.sql.types as T
import pyspark.sql.functions as F

StatementMeta(sasesssparkpool, 11, 1, Finished, Available)

In [9]:
schema = T.StructType(
    [
        T.StructField('SequenceNumber', T.LongType(), False),
        T.StructField('Offset', T.StringType(), False),
        T.StructField('EnqueuedTimeUtc', T.StringType()),
        T.StructField('SystemProperties', T.MapType(T.StringType(), T.StructType([
            T.StructField('member0', T.LongType()),
            T.StructField('member1', T.DoubleType()),
            T.StructField('member2', T.StringType()),
            T.StructField('member3', T.BinaryType()),
        ]))),
        T.StructField('Properties', T.MapType(T.StringType(), T.StructType([
            T.StructField('member0', T.LongType()),
            T.StructField('member1', T.DoubleType()),
            T.StructField('member2', T.StringType()),
            T.StructField('member3', T.BinaryType()),
        ]))),
        T.StructField('Body', T.BinaryType()),
        T.StructField('date', T.DateType()),
        T.StructField('hour', T.IntegerType()),
    ]
)

StatementMeta(sasesssparkpool, 11, 2, Finished, Available)

In [10]:
bodySchema = T.StructType([
    T.StructField("eventId", T.StringType(), False),
    T.StructField("complexData", T.StructType([ T.StructField(f"moreData{i}", T.DoubleType(), False) for i in range(22) ])),
    T.StructField("value", T.StringType(), False),
    T.StructField("type", T.StringType(), False),
    T.StructField("deviceId", T.StringType(), False),
    T.StructField("deviceSequenceNumber", T.LongType(), False),
    T.StructField("createdAt", T.TimestampType(), False)
])

StatementMeta(sasesssparkpool, 11, 3, Finished, Available)

In [11]:
stream_df = read_stream = spark \
    .readStream \
    .schema(schema) \
    .format('avro') \
    .load(folder_path)

normalized_df = stream_df \
        .withColumn('storedAt', F.current_timestamp()) \
        .withColumn('enqueuedAt', F.to_timestamp('EnqueuedTimeUtc', 'M/d/y h:mm:ss a')) \
        .withColumn('Body', F.from_json(F.decode('Body', 'UTF-8'), bodySchema)) \
        .select(col('Body.*'), 'storedAt', 'enqueuedAt')

schema_location = '/datalake'
table_name = 'normalized.device'
output_path = os.path.join(schema_location, 'normalized', 'streaming_device')
checkpoint_path = os.path.join(schema_location, 'checkpoint', 'streaming_device_test')
normalized_df \
    .writeStream \
    .outputMode("append") \
    .format('delta') \
    .option('path', output_path) \
    .option("checkpointLocation", checkpoint_path) \
    .trigger(once=True) \
    .start()

StatementMeta(sasesssparkpool, 11, 4, Finished, Available)